In [1]:
import os.path
from scipy import *
import numpy as np
from numpy import *
from numpy import linalg as LA
from scipy import linalg as LA2
import sympy as sympy
import sys as sys
import time
import matplotlib.pyplot as plt
import itertools as it
from IPython.core.display import HTML


sys.path.append('/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/')
from Modules import BasicTools as BT
from Modules import WaveTools as WT
from Modules import PlotTools as PT
from Modules import FFTTools as FFTT
from Modules import OperatorTools as OT
from Modules import GridTransferTools as GTT
from Modules import TestTools as TT
from Modules import SolverTools as ST
from Modules import WaveformTools as WFT

display(HTML("<style>pre { white-space: pre !important; }</style>"))
np.set_printoptions( linewidth = 10000, threshold = 100000)

In [2]:
nh_min = 32
refCells = 13
refRatio = 2

In [3]:
omega = BT.Grid(nh_min)
finehalf = list(np.arange(refCells))
# finehalf = list(np.arange(int(nh / refRatio)) + int(nh / refRatio))
# print(finehalf)
omega.AddPatch(refRatio, finehalf)
nh_max = omega.nh_max
degFreed = omega.degFreed
waves = WT.MakeWaves(omega)
# PT.PlotWaves(omega, waves = waves)
t = 0
# print(waves)


In [4]:
# ----------------------------------------------------------------------------------------------------------------
# Function: FindSelect
# ----------------------------------------------------------------------------------------------------------------
# By: Sasha Curcic
#
# This function takes in an AMR grid along with the full set of nh_max many Fourier wavevectors, each of degFreed
# length, and creates an operator which will select out degFreed many Fourier modes which do not contain a
# singularity. The precedence rules here are smooth modes and first oscillatory mode, then last mode if needed,
# then wavenumber pairs.
# ----------------------------------------------------------------------------------------------------------------
# Input:
#
# omega               BT.Grid             Grid object
# waves               (N x M) real        Matrix of AMR mode vectors
# ----------------------------------------------------------------------------------------------------------------
# Output:
#
# WTWinv              (M x M) real        A positive non negative number given by the user
# ----------------------------------------------------------------------------------------------------------------

In [5]:
def FindSelect(omega, waves):
    
    errorLoc = 'ERROR:\nOperatorTools:\nFindSelect:\n'
    nhs = omega.nh
    nh_min = omega.nh_min
    nh_max = omega.nh_max
    nh_mid = nhs[-2]
    degFreed = omega.degFreed
    
    # Create selection matrix to be filled.
    fixWaves = np.zeros((nh_max, degFreed), float)

    # Calculate number of modes needed in addition to smooth modes and first oscillatory mode.
    addOn = degFreed - nh_mid - 1
    
    # Calculate start and stop indices of modes through which to iterate and test.
    mode = nh_mid + 1
    wavenum = int((mode / 2) + 1)
    iters = int((nh_max / 2))
    j = mode
    
    # Fill out smooth modes and first oscillatory mode as default.
    np.fill_diagonal(fixWaves[:, :-addOn], 1) # This won't work for low degFreed.
    
    print('Selection Matrix Before Iterating Through Pairs:')
    print(fixWaves)
    print('\n')
    
    final = degFreed
    
    # Iterate through number of wavenumbers left to fill selection matrix.
    for i in range(wavenum, iters):
        
        print('Iteration: ' + str(i) + '\n')
        
        fixWavesTemp = fixWaves + 0
        
        # If any more modes are needed, run through precedence procedure.
        if (addOn > 0):
            
            # If odd number of modes needed, add last mode first.
            if (addOn % 2 == 1):
                fixWavesTemp[-1, -1] = 1
                addOn = addOn - 1
                final = final - 1
            
            # Else, iterate through mode pairs, and stop when the selection operator produced doesn't return a nullspace when applied to the Fourier matrix.
            else:
                nullsize = 1
                remove = np.arange(j+2, final)
                print('remove:', remove)
                while (nullsize > 0):
                    fixWavesTemp = fixWaves + 0
                    fixWavesTemp[mode, j] = 1
                    fixWavesTemp[mode + 1, j + 1] = 1
                    wavesTemp = waves @ fixWavesTemp
#                     print(wavesTemp)
#                     print('')
                    wavesTemp = np.delete(wavesTemp, remove, 1)
#                     print(wavesTemp)
                    nullspace = LA2.null_space(wavesTemp)
                    nullsize = np.shape(nullspace)[1]
                     
                    print('TESTING MODES ' + str(mode) + ' & ' + str(mode + 1) + ':\n')
                    print('Test Selection Matrix:')
                    print(fixWavesTemp)
                    print('')
                    print('Nullspace of Modes Selected:')
                    print(nullspace)
                    print('')
                    print('Size of Nullspace: ' + str(nullsize) + '\n')

                    # ADD AN ERROR CATCH HERE FOR IN CASE YOU CAN'T MANAGE TO SPAN THE SPACE!!!
                    
                    mode = mode + 2
                j = j + 2
                addOn = addOn - 2
            fixWaves = fixWavesTemp
        else:
            break
        print('Successful Matrix Selected:')
        print(fixWaves)
        print('\n')
#     if (degFreed < nh[])
    print('')
    print('Final Selection Matrix:')
    print(fixWaves)
    return fixWaves

In [6]:
# def FindSelect1(omega, waves):
    
#     errorLoc = 'ERROR:\nOperatorTools:\nFindSelect:\n'
#     nhs = omega.nh
#     nh_min = omega.nh_min
#     nh_max = omega.nh_max
#     nh_mid = nhs[-2]
#     degFreed = omega.degFreed
    
#     # Create selection matrix to be filled.
#     fixWaves = np.zeros((nh_max, degFreed), float)

#     # Calculate number of modes needed in addition to smooth modes and first oscillatory mode.
#     addOn = degFreed - nh_mid - 1
    
#     # Calculate start and stop indices of modes through which to iterate and test.
#     mode = nh_mid + 1
#     wavenum = int((mode / 2) + 1)
#     iters = int((nh_max / 2))
#     j = mode
    
#     # Fill out smooth modes and first oscillatory mode as default.
#     np.fill_diagonal(fixWaves[:, :-addOn], 1) # This won't work for low degFreed.
    
#     print('Selection Matrix Before Iterating Through Pairs:')
#     print(fixWaves)
#     print('\n')
    
#     # Iterate through number of wavenumbers left to fill selection matrix.
#     for i in range(mode, nh_max):
        
#         print('Iteration: ' + str(i) + '\n')
        
#         fixWavesTemp = fixWaves + 0
        
#         # If any more modes are needed, run through precedence procedure.
#         if (addOn > 0):
#             nullsize = 1
#             while (nullsize > 0):
#                 fixWavesTemp = fixWaves + 0
#                 fixWavesTemp[mode, j] = 1
#                 wavesTemp = waves @ fixWavesTemp
                
#                 nullspace = LA2.null_space(wavesTemp[:, :j])
#                 nullsize = np.shape(nullspace)[1]

#                 print('TESTING MODE ' + str(mode) + ':\n')
#                 print(wavesTemp[:, :j])
#                 print('Test Selection Matrix:')
#                 print(fixWavesTemp[:, :j])
#                 print('')
#                 print('Nullspace of Modes Selected:')
#                 print(nullspace)
#                 print('')
#                 print('Size of Nullspace: ' + str(nullsize) + '\n')

#                 # ADD AN ERROR CATCH HERE FOR IN CASE YOU CAN'T MANAGE TO SPAN THE SPACE!!!

#                 mode = mode + 2
#             j = j + 1
#             addOn = addOn - 1
#             fixWaves = fixWavesTemp
#         else:
#             break
#         print('Successful Matrix Selected:')
#         print(fixWaves)
#         print('\n')
# #     if (degFreed < nh[])
#     print('')
#     print('Final Selection Matrix:')
#     print(fixWaves)
#     return fixWaves

In [7]:
fixWaves = FindSelect(omega, waves)

Selection Matrix Before Iterating Through Pairs:
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.

In [8]:
for i in range(2, 6):
    print(i)

2
3
4
5


In [9]:
a = 0
A = np.asarray([[1, 1, 0, 0, a], [3, 3, 0, 0, a], [0, 0, 1, 0, a], [0, 0, 1, 0, a], [0, 0, 0, 0, a], [0, 0, 0, 0, a]])
nullspace = LA2.null_space(A, rcond = 1e-9)
rank = LA.matrix_rank(A, tol = 1e-11)
print(A)
print(type(A))
print(nullspace)
print(rank)

[[1 1 0 0 0]
 [3 3 0 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
<class 'numpy.ndarray'>
[[-0.70710678  0.          0.        ]
 [ 0.70710678  0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          1.        ]
 [ 0.          1.          0.        ]]
2


In [10]:
B = np.arange(30).reshape(5, 6)
D = np.arange(3, 5)
C = np.delete(B, D, 1)

print(B)
print('')
print(B[:, :3])
print('')
print(B[:, [0, 1, 2, 5]])
print('')
print(C)
print('')
print(D)
# print(B[:, [:3, 5]])

[[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]
 [12 13 14 15 16 17]
 [18 19 20 21 22 23]
 [24 25 26 27 28 29]]

[[ 0  1  2]
 [ 6  7  8]
 [12 13 14]
 [18 19 20]
 [24 25 26]]

[[ 0  1  2  5]
 [ 6  7  8 11]
 [12 13 14 17]
 [18 19 20 23]
 [24 25 26 29]]

[[ 0  1  2  5]
 [ 6  7  8 11]
 [12 13 14 17]
 [18 19 20 23]
 [24 25 26 29]]

[3 4]


In [11]:
A = np.arange(12)
B = A[:-1] - A[1:]
print(A)
print(B)

[ 0  1  2  3  4  5  6  7  8  9 10 11]
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
